In [ ]:
#@title Copyright 2024 Google LLC. { display-mode: "form" }
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
%%file requirements.txt
google-auth
google-cloud-storage
functions-framework
google-cloud-pubsub
cbers4asat

In [ ]:
%%capture
!pip install -r requirements.txt

In [ ]:
%%writefile main.py
import functions_framework
import google.auth
import io
import requests
import urllib
import json
import base64

from cbers4asat import Cbers4aAPI
from datetime import datetime
from datetime import timedelta

from google.cloud import pubsub_v1

import urllib
import google.auth.transport.requests
import google.oauth2.id_token

from urllib.parse import urlparse
from urllib.parse import urlunparse
import pathlib
import logging
import re

credentials, project_id = google.auth.default();

bucket = "earthengine-test" # @param {"type":"string"}
email = "mdewitt@google.com" # @param {"type":"string"}
api = Cbers4aAPI(email)

# Instantiates a Pub/Sub client
publisher = pubsub_v1.PublisherClient()

@functions_framework.cloud_event
def fetch_data_for_event(cloud_event):
  event_data = cloud_event.data['message']['data']
  print(f"Working with event: {event_data}")
  decoded_str = base64.b64decode(event_data).decode('utf-8')
  message = json.loads(decoded_str)
  properties = message['properties']
  print(trigger_downloads(
      message['bbox'],
      properties['fromdate'],
      properties['todate']))

def trigger_downloads(bounding_box, time_start, time_end):
  time_start = datetime.fromisoformat(time_start)
  time_end = datetime.fromisoformat(time_end)

  topic_name = 'cbers-data-to-download' # @param {"type":"string"}
  # References an existing topic
  topic_path = publisher.topic_path(project_id, topic_name)

  # Example from https://github.com/gabriel-russo/cbers4asat.

  time_window = timedelta(days=7)
  result_collection = api.query(location=bounding_box,
                      initial_date=time_start - time_window,
                      end_date=time_end + time_window,
                      cloud=100,
                      limit=10,
                      collections=['CBERS4A_MUX_L2_DN'])
  features = result_collection['features']
  for feature in features:
    for asset in feature['assets']:
      asset_url = feature['assets'][asset]['href']
      properties = feature['properties']

      # Get the last part of the URL as the filename.
      parsed = urlparse(asset_url)
      asset_path = pathlib.Path(parsed.path)

      path = properties['path']
      row = properties['row']
      im_name = re.sub('_BAND\d+$', '', asset_path.stem)
      dest_path = f"{path}/{row}/{im_name}/{asset_path.name}"
      data = {
          'source_url': asset_url,
          'dest_bucket': bucket,
          'dest_path': dest_path
      }
      print(data)
      message_data = json.dumps(data).encode('utf-8')
      # Publishes a message
      try:
        publish_future = publisher.publish(topic_path, data=message_data)
        publish_future.result()  # Verify the publish succeeded
      except Exception as e:
        logging.error(f"Error publishing event {e}")

  print(f"Triggered {len(features)} image downloads")

Overwriting main.py


In [ ]:
import main
import importlib
importlib.reload(main) # Force reload of the main file.

from google.colab import auth
from cloudevents.http import CloudEvent

PROJECT_ID = "mdewitt-earthengine" # @param {"type":"string"}
auth.authenticate_user(project_id=PROJECT_ID)

attributes = {
    "type": "com.example.sampletype1",
    "source": "https://example.com/event-producer",
}
properties = {
    'bbox': [-46.702, -23.577, -46.676, -23.595],
    'properties': {
        'fromdate': '2024-08-01',
        'todate': '2024-09-11',
    }
}
encoded_properties = base64.b64encode(json.dumps(properties).encode('utf-8'))
data = {'message': {'data': encoded_properties}}
event = CloudEvent(attributes, data)

print(main.fetch_data_for_event(event))

In [ ]:
!gcloud config set functions/region us-east1
!gcloud functions deploy fetch_data_for_event\
  --runtime python312\
  --trigger-topic=brazil-fire-events\
  --project $PROJECT_ID\
  --gen2